# Implementando uma MLP básica com Grid Search para escolher melhor os parâmetros no Mapa de Hénon

## 1. Importando as bibliotecas necessárias

### 1.1 Bibliotecas gerais

In [3]:
%matplotlib widget
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns # a biblioteca 'seaborn' contém vários estilos para os gráficos do 'matpĺotlib'

# agora, melhoramos a qualidade de saida e de visualizacao da imagem 
# alem de mudar a fonte padrao para uma do latex
sns.set_style("ticks")
plt.rcParams['savefig.dpi'] = 200
plt.rcParams["figure.dpi"] = 150

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Palatino"],
})
plt.style.use('dark_background')

### 1.2 Bibliotecas para MLP

In [4]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')

from tensorflow import keras

In [5]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

In [1]:
from pibic2020.models import mlp_model

### 1.3 Bibliotecas dos sistemas caóticos

In [2]:
from pibic2020.tools import timeseries as times
from pibic2020.data import henon as hmap

## 2. Gerando a série temporal do mapa de Hénon

Os valores para os parâmetros das equações serão os "básicos" apresentados no *paper* original.

In [8]:
a = 1.4
b = 0.3
x_inicial = 1.0
y_inicial = 0.0
n_inicial = 0

estados = np.array([x_inicial, y_inicial, n_inicial])

mapa = hmap.MapaHenon(estado_inicial=estados)

In [9]:
n_iteracoes = 5000
estados = mapa.simular(n_iteracoes)

In [10]:
x = estados[:, 0]
y = estados[:, 1]
n = estados[:, 2]

In [11]:
fig, ax = plt.subplots()
ax.plot(n, x, color='LimeGreen', linewidth=0.9)

ax.set_title("100 primeiras iterações da série temporal em $\hat{x}$ do Mapa de Hénon para\n $a =$ " + str(a) + " e $b =$ " + str(b) + " com $x[0] =$ " + str(x[0]) + " e $y[0] =$ " + str(y[0]))
ax.set_ylabel('$x[n]$')
ax.set_xlabel('$n$')
ax.set_xlim(0, 100)  
ax.grid(True)

fig.tight_layout()
sns.despine()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### 2.1 Dividindo em um conjunto de treinamento e de teste, para K = 4 e L = 3

In [12]:
K = 4
L = 3
tam_teste = 0.15

In [13]:
serie_temporal = times.SerieTemporal(x, K=K, L=L)

In [15]:
X_treino, X_teste, y_treino, y_teste = serie_temporal.dividir_treino_teste(tam_teste)

## 3. Definindo o modelo para a MLP

### 3.1 Definindo função para criar a MLP

In [29]:
mlp_gridsearch = mlp_model.ModeloMLP(input_size=K)

Utilizaremos os seguintes parâmetros no *Grid Search*:

In [16]:
param_grid = dict(batch_size=[2, 4, 8, 16, 32], 
                  batch_normalization=['ON', 'OFF'], 
                  activation=['selu', 'relu', 'elu', 'sigmoid', 'tanh'], 
                  init_mode = ['lecun_uniform', 'lecun_normal', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform'],
                  n_neurons = [5, 10, 15, 20, 30, 50, 75, 100],
                  learning_rate = [0.001, 0.003, 0.005, 0.008, 0.01])

Para facilitar, dividiremos esse processo em etapas.

### 3.2 Definindo parâmetros que não serão definidos pelo *Grid Search*

In [17]:
loss = "mean_squared_error"
optimizer = keras.optimizers.Nadam()

### 3.3 Definindo e executando o primeiro *Grid Search*

Primeiro, avaliaremos o impacto do *batch size* e da camada de *batch normalization*.

In [22]:
param_grid_1 = dict(batch_size=[2, 4, 8, 16, 32], 
                  batch_normalization=['ON', 'OFF'])

In [23]:
model_cv_1 = KerasRegressor(build_fn=mlp_gridsearch.gridsearch, epochs=100, verbose=0)

In [24]:
grid = GridSearchCV(estimator=model_cv_1, param_grid=param_grid_1, n_jobs=1, cv=4, scoring='neg_mean_squared_error', verbose=1)

In [25]:
grid_result = grid.fit(X_treino, y_treino)

In [26]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: -0.031684 using {'batch_normalization': 'OFF', 'batch_size': 8}


In [27]:
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

-0.531136 (0.081344) with: {'batch_normalization': 'ON', 'batch_size': 2}
-0.362861 (0.005263) with: {'batch_normalization': 'ON', 'batch_size': 4}
-0.263085 (0.008577) with: {'batch_normalization': 'ON', 'batch_size': 8}
-0.168302 (0.011296) with: {'batch_normalization': 'ON', 'batch_size': 16}
-0.123444 (0.023527) with: {'batch_normalization': 'ON', 'batch_size': 32}
-0.041960 (0.012089) with: {'batch_normalization': 'OFF', 'batch_size': 2}
-0.033340 (0.013276) with: {'batch_normalization': 'OFF', 'batch_size': 4}
-0.031684 (0.008750) with: {'batch_normalization': 'OFF', 'batch_size': 8}
-0.036820 (0.010447) with: {'batch_normalization': 'OFF', 'batch_size': 16}
-0.085695 (0.019583) with: {'batch_normalization': 'OFF', 'batch_size': 32}


### 3.4 Definindo e executando o segundo *Grid Search*

Agora, avaliaremos o impacto do *learning rate* do otimizador.

In [18]:
model_cv_2 = KerasRegressor(build_fn=mlp_gridsearch.gridsearch, epochs=100, verbose=0, batch_size=8, batch_normalization='OFF')

In [19]:
param_grid_2 = dict(learning_rate=[0.001, 0.003, 0.005, 0.008, 0.01])

In [20]:
grid = GridSearchCV(estimator=model_cv_2, param_grid=param_grid_2, n_jobs=1, cv=4, scoring='neg_mean_squared_error', verbose=1)

In [21]:
grid_result = grid.fit(X_treino, y_treino)

In [22]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: -0.015915 using {'learning_rate': 0.003}


In [23]:
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

-0.040259 (0.002800) with: {'learning_rate': 0.001}
-0.015915 (0.004685) with: {'learning_rate': 0.003}
-0.024764 (0.004251) with: {'learning_rate': 0.005}
-0.032914 (0.006057) with: {'learning_rate': 0.008}
-0.107939 (0.120647) with: {'learning_rate': 0.01}


### 3.5 Definindo e executando o terceiro *Grid Search*

Agora, avaliaremos o impacto da função de ativação da camada intermediária.

In [25]:
model_cv_3 = KerasRegressor(build_fn=mlp_gridsearch.gridsearch, epochs=100, verbose=0, batch_size=8, batch_normalization='OFF', learning_rate=0.003)

In [26]:
param_grid_3 = dict(activation=['selu', 'relu', 'elu', 'sigmoid', 'tanh'])

In [27]:
grid = GridSearchCV(estimator=model_cv_3, param_grid=param_grid_3, n_jobs=1, cv=4, scoring='neg_mean_squared_error', verbose=1)

In [28]:
grid_result = grid.fit(X_treino, y_treino)

Fitting 4 folds for each of 5 candidates, totalling 20 fits


In [29]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: -0.002688 using {'activation': 'sigmoid'}


In [30]:
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

-0.025435 (0.010703) with: {'activation': 'selu'}
-0.017485 (0.010364) with: {'activation': 'relu'}
-0.016884 (0.010223) with: {'activation': 'elu'}
-0.002688 (0.000557) with: {'activation': 'sigmoid'}
-0.003098 (0.001063) with: {'activation': 'tanh'}


### 3.6 Definindo e executando o quarto *Grid Search*

Agora, avaliaremos o impacto do inicializador da camada intermediária.

In [31]:
model_cv_4 = KerasRegressor(build_fn=mlp_gridsearch.gridsearch, epochs=100, verbose=0, batch_size=8, batch_normalization='OFF', learning_rate=0.003, activation='sigmoid')

In [32]:
param_grid_4 = dict(init_mode = ['glorot_uniform', 'glorot_normal'])

In [34]:
grid = GridSearchCV(estimator=model_cv_4, param_grid=param_grid_4, n_jobs=1, cv=4, scoring='neg_mean_squared_error', verbose=1)

In [35]:
grid_result = grid.fit(X_treino, y_treino)

Fitting 4 folds for each of 2 candidates, totalling 8 fits


In [36]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: -0.002853 using {'init_mode': 'glorot_normal'}


In [37]:
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

-0.005647 (0.005858) with: {'init_mode': 'glorot_uniform'}
-0.002853 (0.000296) with: {'init_mode': 'glorot_normal'}


### 3.7 Definindo e executando o quinto *Grid Search*

Agora, avaliaremos o número de neurônios na camada intermediária.

In [38]:
model_cv_5 = KerasRegressor(build_fn=mlp_gridsearch.gridsearch, epochs=100, verbose=0, batch_size=8, batch_normalization='OFF', learning_rate=0.003, activation='sigmoid', init_mode='glorot_normal')

In [39]:
param_grid_5 = dict(n_neurons = [5, 10, 15, 20, 30, 50, 75, 100])

In [40]:
grid = GridSearchCV(estimator=model_cv_5, param_grid=param_grid_5, n_jobs=1, cv=4, scoring='neg_mean_squared_error')

In [41]:
grid_result = grid.fit(X_treino, y_treino)

In [42]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: -0.002392 using {'n_neurons': 50}


In [43]:
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

-0.070237 (0.028566) with: {'n_neurons': 5}
-0.016834 (0.005783) with: {'n_neurons': 10}
-0.008536 (0.005339) with: {'n_neurons': 15}
-0.009367 (0.005866) with: {'n_neurons': 20}
-0.003289 (0.001008) with: {'n_neurons': 30}
-0.002392 (0.001208) with: {'n_neurons': 50}
-0.004312 (0.001394) with: {'n_neurons': 75}
-0.003799 (0.002850) with: {'n_neurons': 100}


### Treino com o melhor modelo

In [21]:
model = criar_modelo(name='MLP-basica-henon', batch_normalization='OFF', learning_rate=0.003, activation='sigmoid', init_mode='glorot_normal', n_neurons=50)

In [22]:
model.summary()

Model: "MLP-basica-henon"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
camada_de_entrada (Dense)    (None, 4)                 20        
_________________________________________________________________
camada_intermediaria (Dense) (None, 50)                250       
_________________________________________________________________
camada_de_saida (Dense)      (None, 1)                 51        
Total params: 321
Trainable params: 321
Non-trainable params: 0
_________________________________________________________________


In [23]:
X_treino, X_val, y_treino, y_val = train_test_split(X_treino, y_treino, test_size=0.1)

In [24]:
early_stopping = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True, monitor='val_loss')

In [25]:
batch_size = 8

In [26]:
history = model.fit(X_treino, y_treino, epochs=100,
                            callbacks=early_stopping, validation_data=(X_val, y_val),
                            batch_size=batch_size)
treinamento = pd.DataFrame(history.history)

Epoch 1/100
478/478 [==============================] - 1s 2ms/step - loss: 0.4812 - val_loss: 0.4085
Epoch 2/100
478/478 [==============================] - 1s 1ms/step - loss: 0.4341 - val_loss: 0.4168
Epoch 3/100
478/478 [==============================] - 1s 1ms/step - loss: 0.4180 - val_loss: 0.4297
Epoch 4/100
478/478 [==============================] - 1s 1ms/step - loss: 0.4214 - val_loss: 0.3791
Epoch 5/100
478/478 [==============================] - 1s 1ms/step - loss: 0.4070 - val_loss: 0.3666
Epoch 6/100
478/478 [==============================] - 1s 1ms/step - loss: 0.3647 - val_loss: 0.3493
Epoch 7/100
478/478 [==============================] - 1s 1ms/step - loss: 0.3405 - val_loss: 0.3179
Epoch 8/100
478/478 [==============================] - 1s 1ms/step - loss: 0.3177 - val_loss: 0.2331
Epoch 9/100
478/478 [==============================] - 1s 1ms/step - loss: 0.2274 - val_loss: 0.1859
Epoch 10/100
478/478 [==============================] - 1s 1ms/step - loss: 0.1883 - val_lo

## Teste com o melhor modelo

In [27]:
y_pred = model.predict(X_teste)

### Gráfico

In [33]:
fig, ax = plt.subplots()
ax.plot(n[len(n)-int(len(n)*tam_teste):,], y_teste, color='LimeGreen', label='Valor real')
ax.plot(n[len(n)-int(len(n)*tam_teste):,], y_pred, color='Silver', label='MLP')

ax.set_title("Comparação da predição da MLP com o valor real do mapa de Hénon\n utilizando a rede neural ótima no conjunto de teste")
ax.set_ylabel('$x[n]$')
ax.set_xlabel('$n$')
ax.set_xlim(4900, 5000)
    
ax.grid(True)
sns.despine()
ax.legend()

plt.show()
fig.savefig("../../../../images/mlp-basica/performance/mlp-basica-vs-henon.png")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Erro Quadrático Médio

In [29]:
from sklearn.metrics import mean_squared_error

In [30]:
mse = mean_squared_error(y_teste, y_pred)
print("MSE = " + str(mse))

MSE = 0.003014219721098479


### Salvando o Modelo

In [32]:
model.save("../../../../models/mlp-basica/mlp-basica-henon.h5", include_optimizer=True)